# Multi-GPU Workflows

There are many backends available with CUDA Quantum which enable seamless switching between GPUs, QPUs and CPUs and also allow for workflows involing multiple architectures working in tandem.



In [1]:
import cudaq

targets = cudaq.get_targets()

for target in targets:
     print(target)

Target ionq
	simulator=qpp
	platform=default
	description=

Target default
	simulator=qpp
	platform=default
	description=Default OpenMP CPU-only simulated QPU.

Target nvidia-mqpu-fp64
	simulator=custatevec_fp64
	platform=mqpu
	description=The NVIDIA MQPU FP64 Target provides a simulated QPU for every available CUDA GPU on the underlying system. Each QPU is simulated via cuStateVec FP64.

Target tensornet
	simulator=tensornet
	platform=default
	description=

Target oqc
	simulator=qpp
	platform=default
	description=

Target quantinuum
	simulator=qpp
	platform=default
	description=

Target density-matrix-cpu
	simulator=dm
	platform=default
	description=The Density Matrix CPU Target provides a simulated QPU via OpenMP-enabled, CPU-only density matrix emulation.

Target iqm
	simulator=qpp
	platform=default
	description=

Target nvidia
	simulator=custatevec_fp32
	platform=default
	description=The NVIDIA Target provides a simulated QPU via single-GPU cuStateVec integration on FP32 types.

Ta

### Available Targets

- **default**: The default qpp based CPU backend which is multithreaded to maximise the usage of available cores on your system.

- **nvidia**: GPU based backend which accelerates quantum circuit simulation on NVIDIA GPUs powered by cuQuantum.

- **nvidia-mqpu**: Enables users to program workflows utilizing multiple quantum processors enabled today by GPU emulation. 

- **nvidia-mgpu**: Allows for scaling circuit simulation beyond what is feasible with any QPU today. 

- **density-matrix-cpu**: Noisy simulations via density matrix calculations. CPU only for now with GPU support coming soon. 

Below we explore how to effectively utilize multi-GPU targets.

In [2]:
def ghz_state(qubit_count, target):
    """A function that will generate a variable sized GHZ state (`qubit_count`)."""
    cudaq.set_target(target)

    kernel = cudaq.make_kernel()

    qubits = kernel.qalloc(qubit_count)

    kernel.h(qubits[0])

    for i in range(1, qubit_count):
        kernel.cx(qubits[0], qubits[i])

    kernel.mz(qubits)

    result = cudaq.sample(kernel, shots_count=1000)

    return result

### Default CPU Backend

In [3]:
%%time
cpu_result = ghz_state(qubit_count=2, target="default")

for state,value in cpu_result.items(): print(f'|{state}> : {value}')

|00> : 498
|11> : 502
CPU times: user 1.05 s, sys: 4.25 ms, total: 1.05 s
Wall time: 40.7 ms


### Acceleration via NVIDIA GPUs

Users will notice a **200x speedup** in executing the circuit below on NVIDIA GPUs vs CPUs.

In [4]:
%%time
gpu_result = ghz_state(qubit_count=25, target="nvidia")

for state,value in gpu_result.items(): print(f'|{state}> : {value}')

|0000000000000000000000000> : 484
|1111111111111111111111111> : 516
CPU times: user 509 ms, sys: 384 ms, total: 893 ms
Wall time: 918 ms


### Multiple NVIDIA GPUs

A $n$ qubit quantum state has $2^n$ complex amplitudes, each of which require 8 bytes of memory to store. Hence the total memory required to store a $n$ qubit quantum state is $8$ bytes $\times 2^n$. For $n = 30$ qubits, this is roughly $8$ GB but for $n = 40$, this exponentially increases to 8700 GB. 

If one incrementally increases the qubit count in their circuit, we reach a limit where the memory required is beyond the capabilities of a single GPU. The `nvidia-mgpu` target allows for memory from additional GPUs to be pooled enabling qubit counts to be scaled.  

Execution on the `nvidia-mgpu` backed is enabled via `mpirun`. Users need to create a `.py` file with their code and run the command below in terminal:

`mpirun -np 4 python3 test.py`

where 4 is the number of GPUs one has access to and `test` is the file name chosen.


### Multiple QPU's

The `nvidia-mqpu` backend allows for future workflows made possible via GPU simulation today. 

---

### Asynchronous data collection via batching Hamiltonian terms

Expectation value computations of multi-term hamiltonians can be asynchronously processed via the `mqpu` platform.


<img src="images/hsplit.png" alt="Alt Text" width="500" height="200">


In [5]:
cudaq.set_target("nvidia-mqpu")

qubit_count = 15
term_count = 100000

kernel = cudaq.make_kernel()

qubits = kernel.qalloc(qubit_count)

kernel.h(qubits[0])

for i in range(1, qubit_count):
    kernel.cx(qubits[0], qubits[i])

# We create a random hamiltonian with 10e5 terms
hamiltonian = cudaq.SpinOperator.random(qubit_count, term_count)

# The observe calls allows us to calculate the expectation value of the Hamiltonian, batches the terms, and distributes them over the multiple QPU's/GPUs.

# expectation = cudaq.observe(kernel, hamiltonian)  # Single node, single GPU.

# Single node, multi-GPU.
expectation = cudaq.observe(
    kernel, hamiltonian,
    execution=cudaq.parallel.thread
)

# expectation = cudaq.observe(kernel, hamiltonian, execution= cudaq.parallel.mpi) # Multi-node, multi-GPU.

<font size="4"> Asynchronous data collection via circuit batching</font>

Execution of parameterized circuits with different parameters can be executed asynchronously via the `mqpu` platform.

<img src="images/circsplit.png" alt="Alt Text" width="500" height="200">


In [6]:
import cudaq
from cudaq import spin
import numpy as np

np.random.seed(1)

cudaq.set_target("nvidia-mqpu")

qubit_count = 5
sample_count = 10000
h = spin.z(0)
parameter_count = qubit_count

# Below we run a circuit for 10000 different input parameters.
parameters = (
  np.random
  .default_rng(13)
  .uniform(
    low=0,
    high=1,
    size=(sample_count,parameter_count)
  )
)

kernel, params = cudaq.make_kernel(list)

qubits = kernel.qalloc(qubit_count)
qubits_list = list(range(qubit_count))

for i in range(qubit_count):
    kernel.rx(params[i], qubits[i])

In [7]:
%%time
result = cudaq.observe(kernel, h, parameters)   # Single GPU result.

CPU times: user 3.67 s, sys: 12.4 s, total: 16.1 s
Wall time: 16.3 s


In [8]:
print(f'We have {parameters.shape[0]} parameters which we would like to execute')

# We split our parameters into 4 arrays since we have 4 GPUs available.
xi = np.split(parameters,4)  
N  = [ _xi.shape[0] for _xi in xi ]
print(f'This has been split this into {len(xi)} batches of {N[0]}, {N[1]}, {N[2]}, {N[3]}')

We have 10000 parameters which we would like to execute
This has been split this into 4 batches of 2500, 2500, 2500, 2500


In [9]:
%%time

# Timing the execution on a single GPU vs 4 GPUs, users will see a 4x performance improvement

asyncresults = []

for i in range(len(xi)):
    for j in range(xi[i].shape[0]):
        asyncresults.append(
            cudaq.observe_async(kernel, h, xi[i][j, :], qpu_id=i)
        )

CPU times: user 2.39 s, sys: 1.88 s, total: 4.26 s
Wall time: 1.92 s
